你正在参与祖玛游戏的一个变种。

在这个祖玛游戏变体中，桌面上有 一排 彩球，每个球的颜色可能是：红色 'R'、黄色 'Y'、蓝色 'B'、绿色 'G' 或白色 'W' 。你的手中也有一些彩球。

你的目标是 清空 桌面上所有的球。每一回合：

- 从你手上的彩球中选出 任意一颗 ，然后将其插入桌面上那一排球中：两球之间或这一排球的任一端。
- 接着，如果有出现 三个或者三个以上 且 颜色相同 的球相连的话，就把它们移除掉。
    如果这种移除操作同样导致出现三个或者三个以上且颜色相同的球相连，则可以继续移除这些球，直到不再满足移除条件。
- 如果桌面上所有球都被移除，则认为你赢得本场游戏。
- 重复这个过程，直到你赢了游戏或者手中没有更多的球。

给你一个字符串 board ，表示桌面上最开始的那排球。另给你一个字符串 hand ，表示手里的彩球。请你按上述操作步骤移除掉桌上所有球，计算并返回所需的 最少 球数。如果不能移除桌上所有的球，返回 -1 。

提示：

- 1 <= board.length <= 16
- 1 <= hand.length <= 5
- board 和 hand 由字符 'R'、'Y'、'B'、'G' 和 'W' 组成
- 桌面上一开始的球中，不会有三个及三个以上颜色相同且连着的球

示例：


输入：board = "WRRBBW", hand = "RB"

输出：-1

解释：无法移除桌面上的所有球。可以得到的最好局面是：

\- 插入一个 'R' ，使桌面变为 WRRRBBW 。WRRRBBW -> WBBW

\- 插入一个 'B' ，使桌面变为 WBBBW 。WBBBW -> WW

桌面上还剩着球，没有其他球可以插入。

<br><br>

输入：board = "RBYYBBRRB", hand = "YRBGB"

输出：3

解释：要想清空桌面上的球，可以按下述步骤：

\- 插入一个 'Y' ，使桌面变为 RBYYYBBRRB 。RBYYYBBRRB -> RBBBRRB -> RRRB -> B

\- 插入一个 'B' ，使桌面变为 BB 。

\- 插入一个 'B' ，使桌面变为 BBB 。BBB -> empty

只需从手中出 3 个球就可以清空桌面

核心是在依次将hand球插入board球所有缝隙（有顺序，缝隙可变）的所有方法中，找到最快能消除所有board球的方法，并返回需要的最少hand球数，没有则返回-1；

可暴力搜索，难在优化

方案一：
使用广度优先搜索

三个剪枝条件：

1.当前hand球存在颜色重复球仅判断一次（前提是针对board球每个缝从前往后看一遍hand球-广度优先，每次放球后还要更新hand球）

2.只在当前hand球和board球颜色相同时才放球，并且只放board球前面（那就等价于hand球与board球上一个颜色相同时不放球）

3.只在有更优状况下才继续增加：
- 当前hand球与board球颜色相同
  例如：桌面上的球为 WWRRBBWW，手中的球为 WWRB，答案为 2。
    操作方法如下：WWRRBBWW→WW(R)RRBBWW→WWBBWW→WW(B)BBWW→WWWW→""

- 当前hand球与当前board球颜色不同但当前board球与上一个board球颜色相同
  例如：桌面上的球为 RRWWRRBBRR，手中的球为 WB，答案为 2。
  操作方法如下：RRWWRRBBRR→RRWWRRBBR(W)R→RRWWRR(B)BBRWR→RRWWRRRWR→RRWWWR→RRR→""


In [18]:
from collections import deque
from itertools import product
import re


class Solution:
    def findMinStep(self, board: str, hand: str) -> int:
        def clean(s):
            pattern = r"(.)\1{2,}"
            n=1
            while n:
                s,n = re.subn(pattern,"",s)
            return s

        # hand sort
        hand = "".join(sorted(hand))

        # 广度优先搜索
        queue = deque([(board,hand,0)])
        # visited = {(board,hand)}
        visited=[(board,hand)]

        while queue:
            cur_board,cur_hand,step = queue.popleft()
            # +1是因为末尾也有可能插入
            for i,j in product(range(len(cur_board)+1),range(len(cur_hand))):
                #pruning 1:手上当前球和上一个颜色相同
                if j>0 and cur_hand[j]==cur_hand[j-1]:
                    continue

                #2 只在连续相同颜色的球的开头位置插入新球
                if i>0 and cur_board[i-1]==cur_hand[j]:
                    continue

                #3只在有可能更优结果时插入
                choose = False

                if i<len(cur_board) and cur_hand[j]==cur_board[i]:
                    choose=True
                if 0<i<len(cur_board) and cur_board[i-1]==cur_board[i] and cur_board[i]!=cur_hand[j]:
                    choose=True

                if choose:
                    new_board = clean(cur_board[:i]+cur_hand[j]+cur_board[i:])
                    new_hand = cur_hand[:j]+cur_hand[j+1:]

                    if not new_board:
                        return step+1
                    if (new_board, new_hand) not in visited:
                        queue.append((new_board, new_hand, step + 1))# type: ignore                       
                        # visited.add((new_board, new_hand))
                        visited.append((new_board, new_hand))

        return -1


In [19]:
sol1 = Solution()
print(sol1.findMinStep('rrggbbyywwrrggbb','rgbyw'))

-1


#1.9s-set
#3m 18.3s-list

差距恐怖如斯

In [15]:
list1 = [('rrggbbyywwrrggbb','rgbyw'),('rrggbbyywwrrggbb','rgbyw')]

testboard1='rrggbbyywwrrggbb'
testhand1='rgbyw'
if (testboard1,testhand1) in list1:
    list1.append((testboard1,testhand1))
list1

[('rrggbbyywwrrggbb', 'rgbyw'),
 ('rrggbbyywwrrggbb', 'rgbyw'),
 ('rrggbbyywwrrggbb', 'rgbyw')]

In [9]:
dict1={('rrggbbyywwrrggbb','rgbyw')}


dict